In [ ]:
import os
import json
import requests
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
# read the data
data = pd.read_csv("../results/inf_outf.csv", index_col=0)
# convert index to datetime
data.index = pd.to_datetime(data.index)

In [ ]:
# let's look at inflow/outflow and price
data.drop(columns=["volume", "inflow", "outflow"]).plot()
plt.title("BTC Price and Net Inflow/Outflow")
plt.legend(["Net", "BTC"])
plt.grid(True)

In [ ]:
# calculate the daily change
diff = data.net.diff()
# the standard deviation in daily change
s = diff.std()

# let's observe the variances in change in net in/out
diff.plot.hist(bins=30)
for i in range(3):
    plt.axvline(s*(1+i), color='r')
    plt.axvline(s*(-(i+1)), color='r')
    
plt.title("Standard deviations on close diff")
plt.xticks([s*(-(1+i)) for i in reversed(range(3))] + [s*(1+i) for i in range(3)], rotation=45)
#plt.xlim(-20000, 20000)
plt.show()

In [ ]:
sigma = 3*s      # what we consider anomolous inflow/outflow

# calculating days of large than average net flow
data["net_diff"] = data.net.diff().fillna(0)
data["anom"] = np.where(abs(data.net_diff) > sigma, 1, 0)

print("{:.3f}".format(100*(1-sum(data.anom)/len(data))) + "% non-anomolous days")

In [ ]:
months = mdates.MonthLocator()
monthsFmt = mdates.DateFormatter('%b')
weeks = mdates.WeekdayLocator()

fig, ax = plt.subplots()

ax.plot(data.close, color="goldenrod", lw=1.25)

# add the high inflow/outflow days
for i in data.index[data.anom==1]:
    if data.loc[i].net > 0:
        ax.axvline(i, color='g', alpha=0.7)
    else:
        ax.axvline(i, color='r', alpha=0.7)
    
# set the ticks
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(weeks)

# add grid
ax.grid(True)
ax.set_title("High Inflow/Outflow on BTC Price")
# format ticks
fig.autofmt_xdate()
ax.legend(["BTC"])
plt.show()

## Looking for Whales

In [ ]:
with open("../results/tot_activity.json", "r") as f:
    inp = f.read()
    
act = json.loads(inp)

In [ ]:
def format_activity(d):
    d = json.loads(d)
    df = pd.DataFrame(d).reset_index()
    time = pd.to_numeric(df["index"])
    df.index = [datetime.datetime.fromtimestamp(t//10**3) for t in time]
    df = df.drop(["index"], axis=1)
    return df

In [ ]:
all_activity = {key: format_activity(val) for key, val in act.items() if val}

In [ ]:
infs, outfs = [0]*len(all_activity.keys()), [0]*len(all_activity.keys())
i = 0

for key, val in all_activity.items():
    infs[i] = val.inflow.sum()
    outfs[i] = val.outflow.sum()
    i += 1

In [ ]:
tot_flows = pd.DataFrame({"inf tot":infs, "outf tot":outfs}, index=all_activity.keys())

In [ ]:
tot_flows.sort_values(by=["inf tot"], ascending=False).plot()

In [ ]:
tot_flows["outf tot"].plot()

Look at BitMEX vs. BMEX